In [ ]:
# 1. Before running the code, you will need to download the txt-files.tar.zip 
# file from https://www.gutenberg.org/cache/epub/feeds/
# note: The version of txt-files.tar.zip used for this study was last modified on 2025-03-09 19:18
# note: the full text files are inside of a folder titled epub, but all of the files are .txt files
# note: this file is too large to store in a GitHub repository. It has to be downloaded manually. The
# zip file is 9.9gb, and unzipped the texts are 28.8gb.

# 2. Once downloaded and unzipped, update your directory structure. mine looks like:
# 
# wd/data/
#     ├── pg-texts/
#     │   ├── 1/
#     │   │   ├── pg1.txt
#     │   ├── 2/
#     │   │   ├── pg2.txt
# 
# E.g. wd/data/pg-texts/1/pg1.txt, instead of wd/data/cache/epub/1/pg1.txt
# a) the "epub" folder is renamed to "pg-texts", and b) "pg-texts" is moved
# up a level in the hierarchy, i.e. delete the "cache" folder

In [1]:
import pandas as pd
import csv
import os
import re
import pickle
from collections import Counter

In [2]:
# Load the Project Gutenberg (PG) CSV metadata (download from https://www.gutenberg.org/cache/epub/feeds/)
# The file used for this study was last modified on 2025-02-16 17:04

data = "/path/to/pg_catalog.csv"

In [3]:
# Load PG metadata into dataframe
df = pd.read_csv(data)

In [4]:
# Some texts have multiple Library of Congress classification labels, so we want to split these into
# a separate column to have a more accurate count. Otherwise, the data will contain instances 
# of double classification (e.g. "AB; AC")
# Note: this means these texts will be analyzed twice. that is, a text with "AB; AC"
# will appear in both the AB and AC dataframes

# Parse the LoCC colum. This will put anything after the first semicolon into a new column ("LoCC2")
# And then parses LoCC2, putting anything after the first semicolon into a new column ("LoCC3"), etc, etc

locc_split = df["LoCC"].astype(str).str.split(";", expand=True)

# Rename the columns: "LoCC" for the first label, then "LoCC2", "LoCC3", etc.
df["LoCC"] = locc_split[0]  # Update "LoCC" to contain only the first label
df[["LoCC2", "LoCC3", "LoCC4", "LoCC5"]] = locc_split.iloc[:, 1:5]

In [5]:
df.head()

,Text#,Type,Issued,Title,Language,Authors,Subjects,LoCC,Bookshelves,LoCC2,LoCC3,LoCC4,LoCC5
0,1,Text,1971-12-01,The Declaration of Independence of the United ...,en,"Jefferson, Thomas, 1743-1826","United States -- History -- Revolution, 1775-1...",E201,Politics; American Revolutionary War; United S...,JK,None,None,None
1,2,Text,1972-12-01,The United States Bill of Rights\r\nThe Ten Or...,en,United States,Civil rights -- United States -- Sources; Unit...,JK,Politics; American Revolutionary War; United S...,KF,None,None,None
2,3,Text,1973-11-01,John F. Kennedy's Inaugural Address,en,"Kennedy, John F. (John Fitzgerald), 1917-1963",United States -- Foreign relations -- 1961-196...,E838,Browsing: History - American; Browsing: Politics,None,None,None,None
3,4,Text,1973-11-01,Lincoln's Gettysburg Address\r\nGiven November...,en,"Lincoln, Abraham, 1809-1865",Consecration of cemeteries -- Pennsylvania -- ...,E456,US Civil War; Browsing: History - American; Br...,None,None,None,None
4,5,Text,1975-12-01,The United States Constitution,en,United States,United States -- Politics and government -- 17...,JK,United States; Politics; American Revolutionar...,KF,None,None,None


In [7]:
# Confirm the split worked; LoCC4 should have 4 rows with data; and we should also be able to 
# see the 3 rows with in LoCC5 that contain data

df_sorted_alphabetical = df.sort_values(by="LoCC4", ascending=True)

In [8]:
df_sorted_alphabetical

,Text#,Type,Issued,Title,Language,Authors,Subjects,LoCC,Bookshelves,LoCC2,LoCC3,LoCC4,LoCC5
12195,12266,Text,2004-05-01,Von Haparanda bis San Francisco: Reise-Erinner...,de,"Wasserzieher, Ernst, 1860-1927",Germany -- Description and travel; United Stat...,DD,United States; DE Sachbuch; Browsing: History ...,DK,DL,E151,None
6280,6322,Text,2004-08-01,Personal Narrative of Travels to the Equinocti...,en,"Humboldt, Alexander von, 1769-1859; Bonpland, ...",South America -- Description and travel; Voyag...,G,South America; Browsing: History - General; Br...,QE,QH,F2201,F2301
22421,22492,Text,2007-09-03,Reise in die Aequinoctial-Gegenden des neuen C...,de,"Humboldt, Alexander von, 1769-1859; Hauff, Her...",South America -- Description and travel; Geolo...,G,German Language Books; DE Sachbuch; Browsing: ...,QE,QH,F2201,F2301
24675,24746,Text,2008-03-03,Reise in die Aequinoctial-Gegenden des neuen C...,de,"Humboldt, Alexander von, 1769-1859; Hauff, Her...",South America -- Description and travel; Geolo...,G,DE Sachbuch; Browsing: History - General; Brow...,QE,QH,F2201,F2301
0,1,Text,1971-12-01,The Declaration of Independence of the United ...,en,"Jefferson, Thomas, 1743-1826","United States -- History -- Revolution, 1775-1...",E201,Politics; American Revolutionary War; United S...,JK,None,None,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...
75210,75386,Text,2025-02-16,Kertomuksia ja kuvauksia elämästä,fi,"Kataja, Liina",NaN,nan,NaN,None,None,None,None
75211,75387,Text,2025-02-16,Kohtalon leikkiä,fi,"Röösgrén, Liina",NaN,nan,NaN,None,None,None,None
75212,75388,Text,2025-02-16,Sommerleutnants,de,"Bloem, Walter, 1868-1951",NaN,nan,NaN,None,None,None,None
75213,75389,Text,2025-02-16,Principles and practice of agricultural analysis.,en,"Wiley, Harvey Washington, 1844-1930",NaN,nan,NaN,None,None,None,None


In [5]:
# This counts the total number of times a LoC classification label appears across all
# columns (LoCC, LoCC2, etc). It filters for English-language only; otherwise it includes
# all types of LCC labels (A, A123, AB, AB123, ABC, ABC123, etc)
# The output is label_counts

# Initialize a Counter to tally label instances
label_counts = Counter()

# Iterate through the rows of the DataFrame where Language is "en"
filtered_df = df[df["Language"].fillna("").astype(str).str.strip() == "en"]

# Count occurrences of each label in LoCC, LoCC2, LoCC3, LoCC4, and LoCC5 columns
for col in ["LoCC", "LoCC2", "LoCC3", "LoCC4", "LoCC5"]:
    label_counts.update(filtered_df[col].fillna("").astype(str).str.strip())

# Filter out empty strings from the count
if "" in label_counts:
    del label_counts[""]

# Create a list of labels with more than 100 instances
# labels_list = [label for label, count in label_counts.items() if count > 100]

# Output the list of popular labels and their counts (optional)
# print("Labels with More Than 100 Instances:", labels_list)


In [6]:
label_counts

Counter({'PS': 11658,
         'PR': 9902,
         'PZ': 7297,
         'AP': 2428,
         'DA': 1705,
         'PQ': 1421,
         'PN': 916,
         'DS': 792,
         'PT': 783,
         'D501': 782,
         'BX': 715,
         'BV': 658,
         'QH': 650,
         'QL': 646,
         'BS': 627,
         'DC': 548,
         'E456': 546,
         'Z': 527,
         'E011': 508,
         'D': 499,
         'E151': 454,
         'E300': 448,
         'G': 425,
         'BF': 409,
         'TX': 409,
         'BL': 394,
         'HV': 379,
         'DT': 375,
         'HQ': 366,
         'PA': 348,
         'B': 343,
         'ML': 309,
         'GV': 281,
         'F1001': 276,
         'PG': 272,
         'BJ': 255,
         'AG': 254,
         'GR': 253,
         'ND': 249,
         'DG': 249,
         'PE': 247,
         'CT': 241,
         'SB': 235,
         'SF': 228,
         'BR': 226,
         'nan': 222,
         'DU': 208,
         'Q': 204,
         'M': 195,
     

In [7]:
total_count = sum(label_counts.values())
print(total_count)

62731


In [6]:
# Next, we are going to re-count the labels using the following criteria
# 1. Exclude rows that contain a single letter followed by numbers (D501 and E011 will be excluded)
# 2. Exclude rows that contain only 1 letter (Z will be excluded)
# 3. Include rows that start with 2 letters (PS, DJ, and DJK will be included)
# 4. Combine the count for rows that start with the same first 2 characters (DJK will be added to DJ; QH301 added to QH)
# The output is aggregated_counts

# Filter the counts based on the specified conditions
filtered_counts = Counter({
    label: count for label, count in label_counts.items()
    if len(label) >= 2 and label[:2].isalpha() and not any(char.isdigit() for char in label[:2])  # Handle first two characters correctly
})

# Aggregate counts by the first two characters
aggregated_counts = Counter()
for label, count in filtered_counts.items():
    prefix = label[:2]  # Take the first two characters
    aggregated_counts[prefix] += count

# Print the aggregated counts
print(aggregated_counts)

Counter({'PS': 11658, 'PR': 9902, 'PZ': 7297, 'AP': 2428, 'DA': 1705, 'PQ': 1421, 'PN': 916, 'DS': 792, 'PT': 783, 'BX': 715, 'BV': 658, 'QH': 652, 'QL': 646, 'BS': 627, 'DC': 548, 'BF': 409, 'TX': 409, 'BL': 394, 'HV': 379, 'DT': 375, 'HQ': 366, 'PA': 348, 'ML': 309, 'GV': 281, 'PG': 272, 'BJ': 255, 'AG': 254, 'GR': 253, 'ND': 249, 'DG': 249, 'PE': 247, 'CT': 241, 'SB': 235, 'SF': 228, 'BR': 226, 'na': 222, 'DU': 208, 'RC': 194, 'BT': 191, 'QC': 189, 'NC': 188, 'QK': 188, 'HD': 183, 'TT': 176, 'LB': 163, 'QE': 163, 'NK': 158, 'HE': 157, 'QB': 154, 'QP': 145, 'QA': 141, 'AE': 140, 'BP': 138, 'DH': 133, 'TS': 128, 'GN': 127, 'HX': 119, 'PJ': 118, 'JK': 117, 'SK': 116, 'RA': 116, 'DK': 113, 'TL': 106, 'DP': 106, 'TK': 102, 'DD': 102, 'GT': 98, 'HF': 94, 'UA': 94, 'DF': 89, 'KD': 88, 'PK': 87, 'KF': 86, 'DR': 85, 'RM': 82, 'HB': 81, 'TP': 79, 'QD': 77, 'MT': 75, 'HG': 72, 'TJ': 65, 'JC': 62, 'DL': 62, 'PL': 60, 'HN': 60, 'HT': 59, 'SH': 58, 'LC': 54, 'TA': 52, 'LA': 51, 'HM': 48, 'PH': 48

In [9]:
total_aggregated_count = sum(aggregated_counts.values())
print(total_aggregated_count)

54850


In [8]:
# Now we are going to subtract aggregated_counts from label_counts
# This will tell us how many texts were added into aggregated_counts

# Perform subtraction on keys that are present in both dictionaries
common_keys = set(aggregated_counts.keys()) & set(label_counts.keys())  # Intersection of keys

# Subtract counts for common keys
difference_counts = Counter({
    key: aggregated_counts[key] - label_counts[key] for key in common_keys
})

# Print the resulting difference counts
# For example, 'QH': 2 means that 2 additional texts were added to QH
print(difference_counts)

Counter({'KN': 2, 'DJ': 2, 'QH': 2, 'KD': 1, 'DL': 0, 'HD': 0, 'JV': 0, 'AP': 0, 'KZ': 0, 'JN': 0, 'KL': 0, 'BF': 0, 'BC': 0, 'RF': 0, 'QA': 0, 'AS': 0, 'RB': 0, 'CN': 0, 'BJ': 0, 'LE': 0, 'NK': 0, 'ML': 0, 'LF': 0, 'BH': 0, 'JF': 0, 'RL': 0, 'DQ': 0, 'JS': 0, 'GA': 0, 'PJ': 0, 'PS': 0, 'CJ': 0, 'GV': 0, 'RE': 0, 'QK': 0, 'DC': 0, 'QR': 0, 'RV': 0, 'TN': 0, 'RM': 0, 'JL': 0, 'RA': 0, 'TG': 0, 'HV': 0, 'GF': 0, 'HB': 0, 'DB': 0, 'RC': 0, 'VG': 0, 'BS': 0, 'DF': 0, 'QE': 0, 'PG': 0, 'AE': 0, 'PK': 0, 'QL': 0, 'VF': 0, 'NB': 0, 'NC': 0, 'VE': 0, 'RD': 0, 'LB': 0, 'HM': 0, 'CR': 0, 'GT': 0, 'DU': 0, 'BV': 0, 'DA': 0, 'TS': 0, 'CC': 0, 'PT': 0, 'SK': 0, 'TC': 0, 'TE': 0, 'RK': 0, 'CS': 0, 'BM': 0, 'QM': 0, 'TA': 0, 'HF': 0, 'BP': 0, 'TR': 0, 'GB': 0, 'PF': 0, 'ND': 0, 'HN': 0, 'PL': 0, 'HC': 0, 'PM': 0, 'DD': 0, 'UA': 0, 'RZ': 0, 'CB': 0, 'TX': 0, 'CT': 0, 'AC': 0, 'RJ': 0, 'TT': 0, 'UC': 0, 'UE': 0, 'LH': 0, 'PH': 0, 'KJ': 0, 'PN': 0, 'PQ': 0, 'BR': 0, 'DT': 0, 'DE': 0, 'DK': 0, 'BX': 0, '

In [9]:
# Confirm the count using a different approach: how many texts are there from the original label_counts that
# start with 2 letters (PS) AND contain more 2 characters (PS123 or PSX)

# Filter and print rows that start with 2 letters, contain more than 2 characters, and don't start with numbers
filtered_rows = {
    label: count for label, count in label_counts.items()
    if len(label) > 2 and label[:2].isalpha() and not label[0].isdigit()
}

# Print the filtered rows
# This should correspond with the above difference_counts; QH301: 2 corresponds with QH: 2. KDZ: 1 corresponds with KD: 1, etc
for label, count in filtered_rows.items():
    print(f"{label}: {count}")

KDZ: 1
nan: 222
DJK: 2
QH301: 2
KNX: 2
KBR: 1


In [10]:
# Create a list with the categories containing more than 100 texts (and exclude unclassified NaN/na texts)
# This list will be used to produce the dataframes
# (Instead of hard-coding the two-character categories, create the list from the data)

filtered_keys = [key for key, count in aggregated_counts.items() if count >= 100 and key.lower() != "na"]

# Print the list of filtered keys
print(filtered_keys)

['JK', 'BS', 'PR', 'PS', 'BX', 'PA', 'PE', 'TK', 'PZ', 'QA', 'HX', 'PQ', 'HV', 'NC', 'TL', 'PJ', 'BR', 'BL', 'AE', 'PG', 'AG', 'PT', 'QC', 'DS', 'DA', 'BJ', 'PN', 'CT', 'QH', 'DP', 'GV', 'BF', 'LB', 'ND', 'QL', 'DU', 'BV', 'DG', 'HQ', 'ML', 'DT', 'TX', 'AP', 'BT', 'QK', 'HD', 'DC', 'DK', 'SK', 'QE', 'RC', 'DD', 'QB', 'GR', 'SB', 'BP', 'GN', 'QP', 'HE', 'SF', 'DH', 'RA', 'TS', 'TT', 'NK']


In [19]:
# Print the number of categories to included in the analysis

len(filtered_keys)

65

In [11]:
# Create dictionary to store new dataframes. There will be a new dataframe for each LCC label, and it 
# will contain the metadata for all texts with that label

label_dataframes = {}

In [19]:
# For each LCC label in filtered_keys, go through the PG metadata and look for instances where:
# 1. The LCC label is contained in one of the columns
# 2. Get the text of only those where the language is English 
# If a row matches these criteria, store it in a new dataframe whose name is the LCC label (eg "AB")
# At the end, there will be a dictionary containing 65 dataframes, one dataframe for each label
# that contains more than 100 texts. And the dataframe will contain the metadata for all 100+ records 
# corresponding to that LCC label

matching_rows_count = 0
for label in filtered_keys:
    # filter rows where label matches either "LoCC", "LoCC2", ..., and where "Language" is "en"
    matching_rows = df[
        ((df["LoCC"].fillna("").astype(str).str.strip().eq(label)) |
         (df["LoCC2"].fillna("").astype(str).str.strip().eq(label)) | 
         (df["LoCC3"].fillna("").astype(str).str.strip().eq(label)) |
         (df["LoCC4"].fillna("").astype(str).str.strip().eq(label)) |
         (df["LoCC5"].fillna("").astype(str).str.strip().eq(label))) &
        (df["Language"].fillna("").astype(str).str.strip() == "en")
    ]
    matching_rows_count +=  len(matching_rows)


    
    # Store the resulting dataframe in a dictionary
    label_dataframes[label] = matching_rows

    # Print confirmation
    print(f"Saved {label} with {len(matching_rows)} rows.")

Saved JK with 117 rows.
Saved BS with 627 rows.
Saved PR with 9902 rows.
Saved PS with 11658 rows.
Saved BX with 715 rows.
Saved PA with 348 rows.
Saved PE with 247 rows.
Saved TK with 102 rows.
Saved PZ with 7297 rows.
Saved QA with 141 rows.
Saved HX with 119 rows.
Saved PQ with 1421 rows.
Saved HV with 379 rows.
Saved NC with 188 rows.
Saved TL with 106 rows.
Saved PJ with 118 rows.
Saved BR with 226 rows.
Saved BL with 394 rows.
Saved AE with 140 rows.
Saved PG with 272 rows.
Saved AG with 254 rows.
Saved PT with 783 rows.
Saved QC with 189 rows.
Saved DS with 792 rows.
Saved DA with 1705 rows.
Saved BJ with 255 rows.
Saved PN with 916 rows.
Saved CT with 241 rows.
Saved QH with 650 rows.
Saved DP with 106 rows.
Saved GV with 281 rows.
Saved BF with 409 rows.
Saved LB with 163 rows.
Saved ND with 249 rows.
Saved QL with 646 rows.
Saved DU with 208 rows.
Saved BV with 658 rows.
Saved DG with 249 rows.
Saved HQ with 366 rows.
Saved ML with 309 rows.
Saved DT with 375 rows.
Saved TX w

In [21]:
print(f'Number of total texts: {matching_rows_count}')
print(f'Number of texts with label "QH": {len(label_dataframes["QH"])}')

Number of total texts: 51104
Number of texts with label "QH": 650


In [13]:
# Ensure that the two addition QH301 texts are included
qh301_rows = df[
    (df["LoCC"].fillna("").astype(str).str.strip() == "QH301") &
    (df["Language"].fillna("").astype(str).str.strip() == "en")
]

# Ensure QH301 texts are added to QH
if "QH" in label_dataframes:
    label_dataframes["QH"] = pd.concat([label_dataframes["QH"], qh301_rows], ignore_index=True)
else:
    label_dataframes["QH"] = qh301_rows

matching_rows_count += len(qh301_rows)

In [17]:
print(f'Number of total texts: {matching_rows_count}')
print(f'Number of texts with label "QH": {len(label_dataframes["QH"])}')

Number of total texts 51106
Number of texts with label "QH": 652


In [22]:
# The next steps will add a column to each dataframes that contain the full texts.
# This is going to make the dataframes very large!

# First, define the base file path for where the full text files are stored on local computer
# I.e., define the location of wd/data/pg-texts/
# Note: for Macs, Jupyter Lab prefers the full file path rather than relative path
# I.e. don't use ~/Desktop/.../

BASE_PATH = "/path/to/pg-texts" 

In [23]:
# Create dataset for TF-IDF analysis
# Define a function that will:
# 1. Read full text files from local computer, and
# 2. Perform text preprocessing: for TF-IDF, it is 
#   a) To remove front & end matter from the full text files
#   b) Convert text to lowercase
#   c) Remove punctuation


def load_and_clean_text(text_id):
    file_path = os.path.join(BASE_PATH, str(text_id), f"pg{text_id}.txt")

    if not os.path.exists(file_path):
        return None

    try:
        with open(file_path, "r", encoding="utf-8") as file:
            full_text = file.read()

        # Define regex patterns for front and end matter
        start_pattern = r"\*\*\* START OF (?:THIS|THE) PROJECT GUTENBERG EBOOK .* \*\*\*"
        end_pattern = r"\*\*\* END OF (?:THIS|THE) PROJECT GUTENBERG EBOOK .* \*\*\*"

        # Remove front matter
        start_match = re.search(start_pattern, full_text)
        if start_match:
            full_text = full_text[start_match.end():].strip()

        # Remove end matter
        end_match = re.search(end_pattern, full_text)
        if end_match:
            full_text = full_text[:end_match.start()].strip()

        # Ensure full_text is still a valid string
        if not isinstance(full_text, str):
            raise ValueError("Processed text is not a valid string.")

        # Convert text to lowercase
        full_text = full_text.lower()

        # Remove punctuation using regex (keeps alphanumeric characters and spaces)
        full_text = re.sub(r"[^\w\s]", "", full_text)

        # Format text for CSV: Escape double quotes and replace newlines
        cleaned_text = full_text.replace('"', '""').replace("\n", " ")

        return cleaned_text

    except Exception as e:
        print(f"Error processing {file_path}: {e}")
        return None


In [24]:
# Iterate through each dataframe in the dictionary and read/clean/load data

for label, df in label_dataframes.items():    
    print(f"Processing label: {label}")

    # Apply the function to load and clean texts to each row in the "Text#" column and store 
    # in a new column called "FullText"
    df["FullText"] = df["Text#"].astype(str).apply(load_and_clean_text)

    # Save the updated dataframe back to the dictionary
    label_dataframes[label] = df

Processing label: JK
Processing label: BS
Processing label: PR
Processing label: PS
Processing label: BX
Processing label: PA
Processing label: PE
Processing label: TK
Processing label: PZ
Processing label: QA
Processing label: HX
Processing label: PQ
Processing label: HV
Processing label: NC
Processing label: TL
Processing label: PJ
Processing label: BR
Processing label: BL
Processing label: AE
Processing label: PG
Processing label: AG
Processing label: PT
Processing label: QC
Processing label: DS
Processing label: DA
Processing label: BJ
Processing label: PN
Processing label: CT
Processing label: QH
Processing label: DP
Processing label: GV
Processing label: BF
Processing label: LB
Processing label: ND
Processing label: QL
Processing label: DU
Processing label: BV
Processing label: DG
Processing label: HQ
Processing label: ML
Processing label: DT
Processing label: TX
Processing label: AP
Processing label: BT
Processing label: QK
Processing label: HD
Processing label: DC
Processing la

In [25]:
with open("/path/to/dataPG_TF-IDF.pkl", "wb") as f:
    pickle.dump(label_dataframes, f)